In [ ]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import csv
from io import StringIO
import json
import re

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
src=input()

bangalore


In [ ]:
destn=input()

chennai


In [ ]:
base_url='https://www.goibibo.com/flights/'
base_url = base_url + "/{}-to-{}-flights/"
url = base_url.format(src, destn)

r = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
airline=[]
for i in soup.findAll('span', {'class': 'font13 padL5 black'}):
    airline.append(i.text)
print(airline)
departures=[]
for i in soup.findAll('span', {'class': 'srp-card-uistyles__Time-sc-3flq99-15 hmttrq padT10 fb'}):
    departures.append(i.text)
print(departures)
arrivals=[]
for i in soup.findAll('span', {'class': 'srp-card-uistyles__Time-sc-3flq99-15 hmttrq fb padT10'}):
    arrivals.append(i.text)
print(arrivals)
duration=[]
for i in soup.findAll('span', {'class': 'srp-card-uistyles__DurTime-sc-3flq99-16 erDdYx fb padT10'}):
    duration.append(i.text)
print(duration)
fares = []
for i in soup.findAll('div', {'class': 'srp-card-uistyles__Price-sc-3flq99-17 fOIzpY alignItemsCenter dF fb lh1 padT5'}):
    fares.append(i.text)
print(fares)

['AirAsia India', 'AirAsia India', 'AirAsia India', 'AirAsia India', 'IndiGo', 'IndiGo', 'IndiGo', 'IndiGo', 'IndiGo', 'IndiGo']
['12:50', '15:35', '07:30', '16:25', '22:55', '10:55', '14:50', '15:45', '09:10', '18:15']
['13:45', '16:30', '08:30', '17:25', '23:45', '12:00', '15:55', '17:00', '10:25', '19:15']
['0h 55m', '0h 55m', '1h 00m', '1h 00m', '0h 50m', '1h 05m', '1h 05m', '1h 15m', '1h 15m', '1h 00m']
['1,605', '1,605', '1,605', '1,605', '1,606', '1,606', '1,606', '1,606', '1,606', '1,715']


In [ ]:
import pandas as pd
dkeys = ['duration']
dvalues = [duration]
# print(dvalues)
durationDict =dict(zip(dkeys, dvalues))
print(durationDict)
df = pd.DataFrame(durationDict)
df['duration']=  df['duration'].str.replace("05", "5").str.replace("h", '*60').str.replace(' ','+').str.replace('m','*1').apply(eval)
newDuration = df.values.tolist()
print(newDuration)

{'duration': ['0h 55m', '0h 55m', '1h 00m', '1h 00m', '0h 50m', '1h 05m', '1h 05m', '1h 15m', '1h 15m', '1h 00m']}
[[55], [55], [60], [60], [50], [65], [65], [75], [75], [60]]


In [ ]:
fare_int_arr = []
for i in fares:
  i = re.sub(",","",i)
  i = int(i)
  fare_int_arr.append(i)
print(fare_int_arr)

[1605, 1605, 1605, 1605, 1606, 1606, 1606, 1606, 1606, 1715]


In [ ]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list
duration_final_list = flatten_list(newDuration)

In [ ]:
keys = ['travel-cities','airline','departure', 'arrival', 'duration', 'fares']
travel_cities = src+"-to-"+destn
values = [travel_cities, airline, departures, arrivals, duration_final_list, fare_int_arr]
flightsDict = dict(zip(keys, values))
flightsDict

{'airline': ['AirAsia India',
  'AirAsia India',
  'AirAsia India',
  'AirAsia India',
  'IndiGo',
  'IndiGo',
  'IndiGo',
  'IndiGo',
  'IndiGo',
  'IndiGo'],
 'arrival': ['13:45',
  '16:30',
  '08:30',
  '17:25',
  '23:45',
  '12:00',
  '15:55',
  '17:00',
  '10:25',
  '19:15'],
 'departure': ['12:50',
  '15:35',
  '07:30',
  '16:25',
  '22:55',
  '10:55',
  '14:50',
  '15:45',
  '09:10',
  '18:15'],
 'duration': [55, 55, 60, 60, 50, 65, 65, 75, 75, 60],
 'fares': [1605, 1605, 1605, 1605, 1606, 1606, 1606, 1606, 1606, 1715],
 'travel-cities': 'bangalore-to-chennai'}

Write the scraped content to a json file


In [ ]:
file_name = "/content/gdrive/My Drive/Colab Notebooks/NEW_19OCt/flights1.json"
with open(file_name.format(src, destn), 'a', encoding="utf-8") as outfile:
    json.dump(flightsDict, outfile)
outfile.close()

***Cheap fares***


In [ ]:
doc = lh.fromstring(r.content)
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Kolkata Chennai Cheap Flights"
2:"Fare"
3:"Date"


In [ ]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

cheap_flights_dict={title:column for (title,column) in col}
# df=pd.DataFrame(Dict)
# df.head(5)
print(cheap_flights_dict)

{'Kolkata Chennai Cheap Flights': ['Lowest Airfare Today', 'Cheapest Fare Within 30 Days', 'Cheapest Fare Within 90 Days', 'Cheapest Fare Within 365 Days'], 'Fare': ['Rs 6488', 'Rs 3454', 'Rs 2966', 'Rs 2966'], 'Date': ['10 Oct', '09 Nov', '28 Dec', '28 Dec']}


In [ ]:
file_name = "/content/gdrive/My Drive/Colab Notebooks/NEW_19OCt/{}-to-{}-flights.json".format(src, dest)
with open(file_name.format(src, destn), 'a', encoding="utf-8") as outfile:
    json.dump(cheap_flights_dict, outfile)
outfile.close()